In [12]:
%load_ext autoreload
%autoreload 2
from experiments import *
from cpmpy import *

In [13]:
#instance_level(1)

In [14]:
# above but in cell
t=1

path = f"instances/type{t:02d}/inst*.json"
files = sorted(glob.glob(path))
instances = []
for file in files:
    print(file)
    with open(file) as f:
        instances.append(Instance(int(file.split("/")[-1].split(".")[0][8:]), json.load(f), t))

instances/type01/instance0.json
instances/type01/instance1.json
instances/type01/instance2.json
instances/type01/instance3.json
instances/type01/instance4.json
instances/type01/instance5.json
instances/type01/instance6.json
instances/type01/instance7.json
instances/type01/instance8.json
instances/type01/instance9.json


In [15]:
# Learn propositional models and check their quality
#for i, instance in enumerate(instances):
i = 0
instance = instances[0]
assert(instance.has_solutions())

m, m_vars, _, stats = instance.learn_model(propositional=True)
print(m, m_vars, stats)
#pickle_var[files[i]] = [m, m_vars]
percentage_pos, percentage_neg = instance.check(m, m_vars)
print(percentage_pos, percentage_neg)
#tests_classification = instance.test(m, m_vars)
#print(tests_classification)
#save_results_json(instance.problem_type, instance.number, tests_classification)


Constraints:
    [[abs([(list[6]) + (-(list[8]))]) >= 1], [abs([(list[4]) + (-(list[5]))]) >= 1], [abs([(list[0]) + (-(list[1]))]) >= 1]]
Objective: None [list[0], list[1], list[2], list[3], list[4], list[5], list[6], list[7], list[8], list[9]] {'all_constraints': 9, 'reduced_constraints': 3}
100.0 100.0


In [16]:
m.solve()
[x.value() for x in m_vars]
# !?? what did you do with m_vars... you created new ones in filter_redundant()!! fixed that

[2, 1, 10, 7, 2, 1, 2, 8, 1, 4]

In [17]:
all_data = np.vstack(
    [instance.flatten_data(instance.pos_data), instance.flatten_data(instance.neg_data),
     instance.flatten_data(instance.test_data)])

In [18]:
extra_solutions(m, cpmpy.cpm_array(m_vars), all_data)

[7, 6, 8, 7, 6, 3, 3, 4, 8, 10]
[7, 6, 8, 7, 6, 3, 3, 4, 8, 9]
[7, 6, 8, 7, 6, 3, 4, 4, 8, 9]


[[7, 6, 8, 7, 6, 3, 3, 4, 8, 10],
 [7, 6, 8, 7, 6, 3, 3, 4, 8, 9],
 [7, 6, 8, 7, 6, 3, 4, 4, 8, 9]]

In [19]:
# break down of 'extra_solutions'
existing_sol = all_data
m_vars = cpm_array(m_vars)

m2 = cpmpy.Model([c for c in m.constraints])
print(m2)
print(m2.solve()) # not sure why this is needed
m_vars.value()

Constraints:
    [[abs([(list[6]) + (-(list[8]))]) >= 1], [abs([(list[4]) + (-(list[5]))]) >= 1], [abs([(list[0]) + (-(list[1]))]) >= 1]]
Objective: None
True


array([2, 1, 8, 7, 2, 1, 2, 4, 1, 9])

In [20]:
from cpmpy.transformations.get_variables import get_variables
vs = get_variables(m2.constraints)
cpm_array(vs).value()

array([2, 1, 2, 1, 2, 1])

In [21]:
print(len(existing_sol))
for sol in existing_sol:
    # this would better be a negative table constraint...
    m2 += ~cpmpy.all(m_vars == sol)
m2.solve()

2500


True

In [32]:
# Faster, new style (work in progress)
from cpmpy_helper import *
from cpmpy.solvers import CPM_ortools
s2 = CPM_ortools(m2)

collector = []
def myprint():
    print(m_vars.value())
    collector.append(m_vars.value())
solveAll(s2, display=myprint, solution_limit=20)

print("Got", len(collector), "alternative solutiosn")

[ 1  4  7  1  4 10  4  6 10  5]
[5 6 4 6 3 9 4 4 3 1]
[5 6 7 9 7 6 3 5 4 1]
[ 5  6 10 10  7  6  4  7  3  7]
[3 5 9 1 7 6 4 7 5 5]
[ 4  5 10  1  5  3  3  7  4  1]
[1 2 7 1 8 3 8 1 9 5]
[3 5 7 1 3 1 2 1 3 1]
[ 6  5 10  1  7  4  6  1  5  1]
[ 7  8 10  1  4  3  1  4  2  7]
[4 3 9 1 8 9 2 4 3 7]
[ 4  2  9  1  8  9 10  4  9  7]
[8 4 9 1 7 9 2 4 1 7]
[8 4 9 1 2 1 2 4 1 7]
[8 4 9 1 8 9 3 4 2 7]
[8 4 9 1 2 4 3 4 2 7]
[ 8  4  9  1 10  9  3  4  2  7]
[ 8  4  9  1 10  9  3  4  1  7]
[ 8  4  9  1 10  9  9  4 10  7]
[ 8  4  9  1 10  9  8  4 10  7]
Got 20 alternative solutiosn


In [ ]:
existing_sol[0]